# 1. Imports

In [0]:
%run /Workspace/Users/matheusfelipebasso@hotmail.com/Datum/utils/olist/LOAD_BRONZE_CSV

In [0]:
%run /Workspace/Users/matheusfelipebasso@hotmail.com/Datum/utils/olist/SAFE_SAVE_SILVER

In [0]:
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql.functions import col, trim

---
# 2. Leitura

In [0]:
schema = StructType([
    StructField('customer_id',              StringType(), False),
    StructField('customer_unique_id',       StringType(), False),
    StructField('customer_zip_code_prefix', StringType(), True),
    StructField('customer_city',            StringType(), True),
    StructField('customer_state',           StringType(), True),
])


Como podemos ver acima o schema foi definido como `False` para as colunas `customer_id` e `customer_unique_id`, assim garantindo que o valor delas não está nulo por se tratarem de colunas chave para identificação e relação com as demais.

In [0]:
df_customer = return_df_from_bronze(key_word='_customers_', schema=schema)

In [0]:
df_customer.limit(10).display()

customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,09790,sao bernardo do campo,SP
4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,01151,sao paulo,SP
b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,08775,mogi das cruzes,SP
4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP
879864dab9bc3047522c92c82e1212b8,4c93744516667ad3b8f1fb645a3116a4,89254,jaragua do sul,SC
fd826e7cf63160e536e0908c76c3f441,addec96d2e059c80c30fe6871d30d177,04534,sao paulo,SP
5e274e7a0c3809e14aba7ad5aae0d407,57b2a98a409812fe9618067b6b8ebe4f,35182,timoteo,MG
5adf08e34b2e993982a47070956c5c65,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR
4b7139f34592b3a31687243a302fa75b,9afe194fb833f79e300e37e580171f22,30575,belo horizonte,MG


---
# 3. Análise colunas

In [0]:
for _col in [x for x in df_customer.columns if x not in ['customer_id', 'customer_unique_id']]:
    _col_not_null = df_customer.select(_col).where(col(_col).isNotNull()).distinct().orderBy(_col, ascending=False).count()
    _col_null = df_customer.select(_col).where((col(_col).isNull()) | (trim(col(_col)) == '')).distinct().orderBy(_col, ascending=False).count()
    print(f"NULL: {_col_null} | NOT NULL: {_col_not_null} | {_col}")

NULL: 0 | NOT NULL: 14994 | customer_zip_code_prefix
NULL: 0 | NOT NULL: 4119 | customer_city
NULL: 0 | NOT NULL: 27 | customer_state


---
# 4. Delta Lake

Utilizando `format("delta")` para aproveitar as vantagens da estrutura.

In [0]:
olist_silver_delta_saving(df=df_customer, table_name='olist_customers')

Sucesso para para a tabela "datum.silver.olist_customers"
Tempo de Execução: 00:00:06


In [0]:
del df_customer